In [7]:
import json
import os
from datetime import datetime
from appdirs import user_data_dir


timestamp = datetime.now().strftime("%Y%m%d-%H-%M-%S")
duration_h = 2
algo = 'ori'

use_large = False

app = f"03oos_openwsn_prog_{algo}"
site = "lille"
energy_profile = "energym3"

small_nodes = "238-242+225-229" # 5*2 = 10
small_nodes_root = "240"
small_near_root = "227"

large_nodes = "250-255+238-243+225-230+205-210+191-196" # 6*5 = 30 # 169-174+152-157
large_nodes_root = "207"
large_near_root = "208"

if use_large:
    nodes = large_nodes
    root = large_nodes_root
    near_root = large_near_root
else:
    nodes = small_nodes
    root = small_nodes_root
    near_root = small_near_root

In [8]:
file_name = 'exp_log'
def get_log_path(file_name="x", format="txt", folder_only=False):
    if folder_only:
        return user_data_dir('openvisualizer')
    else:
        return os.path.join(user_data_dir('openvisualizer'), file_name+"."+format)

def run_command(cmd):
    val = !{cmd}
    val = [str(i).strip() for i in val]
    result = json.loads("".join(val))
    return result

### Submit experimet

In [18]:
# Submit experiment
cmd = f"iotlab-experiment submit -n test_{timestamp} -d {int(duration_h*60)} -l {site},m3,{nodes},{algo},{energy_profile}"

print(cmd)

iotlab-experiment submit -n test_20230110-04-56-03 -d 120 -l lille,m3,238-242+225-229,ori,energym3


In [20]:
!iotlab-experiment submit -n test_20230110-04-56-03 -d 120 -l lille,m3,238-242+225-229,ori,energym3

{
    "id": 352756
}


In [21]:
result = run_command(cmd)
exp_id = result["id"]

cmd = f"iotlab-experiment wait -i {exp_id}"
# run_command(cmd)


In [23]:
node_list = []
for group in nodes.split("+"):
    start, end = group.split("-")
    start = int(start)
    end = int(end)
    gr_ = list(range(start,end+1))
    node_list.extend(gr_)

node_id_list = [f"m3-{n}" for n in node_list]
node_site_list = [f"{n}.{site}.iot-lab.info" for n in node_id_list]
node_site_list

['m3-238.lille.iot-lab.info',
 'm3-239.lille.iot-lab.info',
 'm3-240.lille.iot-lab.info',
 'm3-241.lille.iot-lab.info',
 'm3-242.lille.iot-lab.info',
 'm3-225.lille.iot-lab.info',
 'm3-226.lille.iot-lab.info',
 'm3-227.lille.iot-lab.info',
 'm3-228.lille.iot-lab.info',
 'm3-229.lille.iot-lab.info']

In [26]:
cmd = f"openv-server --iotlab-motes {' '.join(node_site_list)}"
cmd

'openv-server --iotlab-motes m3-238.lille.iot-lab.info m3-239.lille.iot-lab.info m3-240.lille.iot-lab.info m3-241.lille.iot-lab.info m3-242.lille.iot-lab.info m3-225.lille.iot-lab.info m3-226.lille.iot-lab.info m3-227.lille.iot-lab.info m3-228.lille.iot-lab.info m3-229.lille.iot-lab.info'

In [27]:
root_site = f"m3-{root}.{site}.iot-lab.info"
cmd = f"openv-client root {root_site}"
cmd

'openv-client root m3-240.lille.iot-lab.info'

In [24]:

cmd = "openv-client view web"

In [ ]:
# download oml energy
oml_path = get_log_path(folder_only=True)
cmd = f"pscp -r -i ~/.ssh/id_rsa.ppk dfawwaz@saclay.iot-lab.info:.iot-lab/{exp_id}/consumption/ {oml_path}/consumption/"
cmd = f"plot_oml_consum -p -i {oml_path}/consumption/m3-{near_root}.oml"


### Add/Remove mote

In [ ]:
# To remove: Stop and clear flash

# To add: Stop and clear flash, then flash with oos and start

last_node = node_list[-1]
cmd = f"iotlab-node --stop -l {site},m3,{last_node}"
cmd = f"iotlab-node --start -l {site},m3,{last_node}"